In [ ]:
!pip install wandb

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback

In [3]:
wandb.init(project='Fashion_Mnist')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
config = wandb.config
config.learning_rate = 0.001

## dataset

In [5]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size= 0.1)

# normalizaition
X_train = X_train / 255.0
X_test = X_test / 255.0

4422102/4422102 [==============================] - 0s 0us/step


## CNN + MLP

In [6]:
model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (5, 5), activation='relu'),
    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [8]:
model.compile(optimizer= tf.keras.optimizers.Adam(config.learning_rate),
              loss= tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy']
)

In [9]:
model.fit(X_train, Y_train, epochs=5, validation_data=(X_val, Y_val), callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5
1687/1688 [============================>.] - ETA: 0s - loss: 0.4962 - accuracy: 0.8192

wandb: Adding directory to artifact (/content/wandb/run-20230714_201840-fa0xveiv/files/model-best)... Done. 0.0s


1688/1688 [==============================] - 68s 39ms/step - loss: 0.4961 - accuracy: 0.8192 - val_loss: 60.3815 - val_accuracy: 0.8110
Epoch 2/5
1686/1688 [============================>.] - ETA: 0s - loss: 0.3232 - accuracy: 0.8812

wandb: Adding directory to artifact (/content/wandb/run-20230714_201840-fa0xveiv/files/model-best)... Done. 0.0s


1688/1688 [==============================] - 62s 37ms/step - loss: 0.3231 - accuracy: 0.8812 - val_loss: 42.4215 - val_accuracy: 0.8218
Epoch 3/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.2706 - accuracy: 0.9011 - val_loss: 53.2099 - val_accuracy: 0.8137
Epoch 4/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.2368 - accuracy: 0.9134 - val_loss: 45.3967 - val_accuracy: 0.8290
Epoch 5/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.2132 - accuracy: 0.9211 - val_loss: 62.7122 - val_accuracy: 0.7922


In [11]:
model.save('CNN_fashion_mnist.h5')

In [12]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.2553 - accuracy: 0.9101


[0.2552800178527832, 0.910099983215332]

## MLP

In [13]:
model = tf.keras.models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [14]:
model.compile(optimizer= 'Adam',
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [15]:
model.fit(X_train, Y_train, epochs=10, callbacks=[WandbCallback()])

Epoch 1/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.5119 - accuracy: 0.8208
Epoch 2/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3865 - accuracy: 0.8616
Epoch 3/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3463 - accuracy: 0.8750
Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3192 - accuracy: 0.8839
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3023 - accuracy: 0.8883
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2877 - accuracy: 0.8944
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2721 - accuracy: 0.8992
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2633 - accuracy: 0.9019
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2517 - accuracy: 0.9072
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.241

In [16]:
model.save('MLP_fashion_mnist.h5')

In [17]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3571 - accuracy: 0.8776


[0.3570767641067505, 0.8776000142097473]